In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from tasks import NER
import torch

In [2]:
# load phobert-ner đã được fine-tuned
phobert_ner = AutoModelForTokenClassification.from_pretrained("huyenbui117/Covid19_phoNER")
# load tokenizer của phobert
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
labels = NER().get_labels(None)
labels

['O',
 'B-ORGANIZATION',
 'O-ORGANIZATION',
 'I-ORGANIZATION',
 'B-LOCATION',
 'I-LOCATION',
 'B-SYMPTOM_AND_DISEASE',
 'I-SYMPTOM_AND_DISEASE',
 'B-DATE',
 'I-DATE',
 'B-PATIENT_ID',
 'I-PATIENT_ID',
 'B-AGE',
 'I-AGE',
 'B-NAME',
 'I-NAME',
 'B-JOB',
 'I-JOB',
 'B-TRANSPORTATION',
 'I-TRANSPORTATION',
 'B-GENDER']

In [4]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [14]:
import pandas as pd
df= pd.DataFrame(columns=["tokens","predictions"])
with open("sentences.txt", 'r', encoding="utf-8") as file:
    for line in file:
        sentences = rdrsegmenter.tokenize(line)
        for sentence in sentences:
            sequence = " ".join(sentence)  # tạo câu mới đã được tokenized

            input_ids = torch.tensor([tokenizer.encode(sequence)])  # lấy id của các tokens tương ứng
            # không dùng tokenize(decode(encode)), text sẽ bị lỗi khi tokenize do conflict với tokenizer mặc định
            tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(sequence))  # lấy các token để đánh tags

            outputs = phobert_ner(input_ids).logits
            predictions = torch.argmax(outputs, dim=2)
            print(predictions)


            for i in [[token, labels[prediction]] for token, prediction in zip(tokens, predictions[0].numpy())]:
                print(i)
                a_series = pd.Series(i, index = df.columns)
                df = df.append(a_series, ignore_index=True)
            print('---------------------------------------------')
print(df)

tensor([[ 0,  0, 20,  0, 10,  0, 12,  0,  0,  0, 16,  0,  0,  1,  3,  0,  0,  6,
          7,  0,  0,  0,  0,  0,  4,  5,  0]])
['<s>', 'O']
['Bệnh_nhân', 'O']
['nữ', 'B-GENDER']
['số', 'O']
['90', 'B-PATIENT_ID']
[',', 'O']
['30', 'B-AGE']
['tuổi', 'O']
['là', 'O']
['một', 'O']
['bán', 'B-JOB']
['bảo_hiểm', 'O']
['tại', 'O']
['Công_ty', 'B-ORGANIZATION']
['Samsung', 'I-ORGANIZATION']
['có', 'O']
['dấu_hiệu', 'O']
['suy', 'B-SYMPTOM_AND_DISEASE']
['hô_hấp', 'I-SYMPTOM_AND_DISEASE']
[',', 'O']
['đang', 'O']
['được', 'O']
['điều_trị', 'O']
['tại', 'O']
['bệnh_viện', 'B-LOCATION']
['Chợ_Rẫy', 'I-LOCATION']
['</s>', 'O']
---------------------------------------------
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          4,  0,  0,  0, 18, 19, 19, 19,  0,  8,  0]])
['<s>', 'O']
['Bệnh_nhân', 'O']
['có', 'O']
['đi', 'O']
['cùng', 'O']
['một', 'O']
['tài_xế', 'O']
['taxi', 'O']
['biển', 'O']
['số', 'O']
[',', 'O']
['là', 'O']
['F@@', 'O']
['0', 'O']
['của',